# Amazon IoT Fleetwise for vision system data collection and transformation

**Note:** AWS IoT FleetWise is currently available in `us-east-1` and `eu-central-1`.

AWS IoT FleetWise is a managed service that you can use to collect vehicle data and organize it in
the cloud. With AWS IoT FleetWise, you can standardize all of your vehicle data models, independent
of the in-vehicle communication architecture. You can also define data collection rules to transfer
only high-value data to the cloud. AWS IoT FleetWise can be used to process collected data, gain
insights about a vehicle's health, and help diagnose and troubleshoot potential issues with your
vehicles.

Additionally, AWS IoT FleetWise provides the capability to collect ECU data and store them on cloud
databases. You can utilize different AWS services (Eg: Amazon QuickSight for analytics and Amazon
SageMaker for machine learning) to develop novel use cases that augment your existing vehicle
functionality. In particular, AWS IoT FleetWise can leverage fleet data (big data) and help you
develop use cases that create business value. For example, AWS IoT FleetWise can be used to improve
electric vehicle range estimation, optimize battery life charging, and optimize vehicle routing.

For more information, see
[What is AWS IoT FleetWise?](https://docs.aws.amazon.com/iot-fleetwise/latest/developerguide/) in
the _AWS IoT FleetWise Developer Guide_.

IoT FleetWise is comprised of two components, an open-source software called the "Reference
Implementation for AWS IoT FleetWise" (FWE) running on Linux-based hardware in the vehicle and the
IoT FleetWise cloud where you can create cloud resources to model, select, and collect vehicle data.

Vision system data collection plays a crucial role in the automotive industry, particularly in the
development of advanced driver assistance systems (ADAS), autonomous vehicles, and vehicle
diagnostics. Different sources of data in the data collection are Sensors like Lidar, Radar, Camera,
Ultrasonic sensors, IMU, GPS, OBD, and more. This data is then used for Data fusion to create a
comprehensive and accurate representation of the vehicle's environment. This is crucial for making
real-time decisions in ADAS and autonomous driving systems.

## Problem statement

Data Variety, Data quality, Compliance and Data Governance are challenges with growing standards in
the industry.

Vision system data extraction is a time consuming task for Original Equipment Manufacturers (OEMs)
and Tier-1s. Scenario-based data collection is a hard requirement for many ADAS related features
like accident detection which requires low latency for vehicle-to-everything (V2X) related use
cases.

## Basics and understanding VSS and Vision System Data

VSS (Vehicle Signal Specification) primarily resembles the physical structure of the vehicle, so
there often isn't a need to repeat branches and data entries (like doors or axles). There isn't one
ADAS camera or controller manufacturer in the automotive industry, and OEMs use different vendors or
the same OEM may use different vendors for different vehicle models. AWS IoT FleetWise provides an
advantage because with VSS standardization, you can bring your own Interface Definition Language
(IDL) environment. An IDL is used to define the interfaces, properties, and behaviors of software
components or systems, allowing for communication and interoperability between different software
modules. You can then have signals defined as per VSS standard, and the signals can be reused with
any fleet.

To demonstrate the key Vision System Data API and data collection functionality, we will use a
standard Robot Operating System (ROS 2) message,
[sensor_msgs/msg/CompressedImage](https://docs.ros2.org/galactic/api/sensor_msgs/msg/CompressedImage.html),
published on the ROS topic `/Cameras/Front/Image`. This message is defined as:

<pre>
<a href="https://docs.ros2.org/galactic/api/std_msgs/msg/Header.html">sensor_msgs/msg/CompressedImage.msg</a>:

   <a href="https://docs.ros2.org/galactic/api/std_msgs/msg/Header.html">std_msgs/msg/Header</a> header
      <a href="https://docs.ros2.org/galactic/api/builtin_interfaces/msg/Time.html">builtin_interfaces/msg/Time</a> stamp
         int32 sec
         uint32 nanosec
      string frame_id
   string format
   uint8[] data
</pre>

The example message contains several data types: strings, int32, uint32, and dynamic-size array of
uint8. It also has the header message nested inside. Vision system data API supports all ROS 2 field
types, including all types of arrays. For more information, see
[https://docs.ros.org/en/galactic/Concepts/About-ROS-Interfaces.html#field-types](https://docs.ros.org/en/galactic/Concepts/About-ROS-Interfaces.html#field-types).
Field default values are ignored by the API modeling and data collection. The constants are not
supported by the API modeling.

**Vision System Data API** enables the collection of data transmitted as **ROS 2 messages**. This
can be data collected from camera, radar, lidar, or other sensors - or any corresponding metadata.

ROS 2 message format is a crucial part of the ROS 2 communication system, enabling different nodes
to exchange data and information in a standardized and efficient manner. It plays a central role in
the inter-process communication that is fundamental to the ROS 2 framework's operation in automotive
applications. This guide will walk you through the vision system data APIs and demonstrate data
collection using an example standard message.

## Understanding Signal Catalog

ROS 2 messages are modeled using four essential VSS components:

- **Branch**: a branch is a tree node that contains other nodes. It can be a simple branch or a
  struct that has children nodes.

- **Sensor**: A sensor is a modeled primitive or complex value that is collected from the vehicle.
  For ROS 2, the ROS topic is modeled as a sensor, such as `/Cameras/Front/Image`.

- **Struct**: A structure (or struct) is a signal that is described by multiple values. For ROS 2,
  the ROS message is modeled as a struct (like `sensor_msgs/msg/CompressedImage.msg`,
  `std_msgs/msg/Header.msg`, or `builtin_interfaces/msg/Time.msg`).

  _Note: A struct can't be a top level member. It must be a member of a branch._

- **Property**: a struct member. For ROS 2, every message field is modeled as a property. It can be
  a primitive data type, such as UINT8, or another struct (like `header`, `stamp`, `sec`, `nanosec`,
  `frame_id`, `format`, or `data`).

Properties have field **dataEncoding**, that can be either `TYPED` or `BINARY`. Only properties of
type `UINT8_ARRAY` and `STRING` can be declared as `BINARY`. If you want to have fields for a ROS 2
message of type `uint8[]` being stored as a binary file that is separate from the rest of the
message, you can declare the fields as `BINARY`. The output JSON or Apache Parquet file contains a
link to the binary file, which is stored in the Amazon S3 bucket as a field value.

### Approach

In this demo, we will start with a sample ROS 2 message and then abstract user from all the internal
calls (Create, Read, Update, Delete) and perform the transformation. The output JSON/Parquet file is
then visualized.

1. Prepare a sample ROS 2 message to demonstrate and understand the API
2. Create a network-interfaces.json file
3. Generate the CreateSignalCatalog API input
4. Generate CreateDecoderManifest API input
5. Create or update a signal catalog
6. Create and activate a model manifest (vehicle model)
7. Create and activate a decoder manifest
8. Create and activate a campaign to collect ROS 2 data
9. Visualize data using Python libraries

## Prerequisites

- Access to an AWS account with administrator privileges.
- Signed in to the AWS IoT FleetWise console in the `us-east-1` Region using the account with
  administrator privileges.
- (Optional) IDL files and their corresponding ROS 2 bag (rosbag) files are available

## Scope

This guide demonstrates the Reference Implementation for AWS IoT FleetWise ("FWE") functionality for
'vision system data' using a [ROS 2 Galactic](https://docs.ros.org/en/galactic/) data source.

The following diagram illustrates the dataflow and artifacts consumed and produced by this demo:

![Vision system data demo diagram](vision-system-data-demo-diagram.jpg)

## Launch your development machine

Use the following instructions to launch an AWS EC2 Graviton (arm64) instance with a jupyter-lab
server, and follow the step-by-step jupyter notebook. Pricing for EC2 can be found,
[here](https://aws.amazon.com/ec2/pricing/on-demand/).

1. Launch an EC2 Graviton instance with administrator permissions:
   [**Launch CloudFormation Template**](https://us-east-1.console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/quickcreate?templateUrl=https%3A%2F%2Faws-iot-fleetwise.s3.us-west-2.amazonaws.com%2Flatest%2Fcfn-templates%2Fvision-system-data-jupyter.yml&stackName=fwdev-jupyter).
1. **Select the checkbox** next to _'I acknowledge that AWS CloudFormation might create IAM
   resources with custom names.'_
1. Choose **Create stack**.
1. Wait until the status of the Stack is **CREATE_COMPLETE**; this can take up to ten minutes.
1. For security purposes, the created stack does not expose the jupyter server to the internet.
   Instead, you will have to create an ssh port forwarding to you EC2 instance from your local
   machine. Execute the following from a local terminal, replacing `<NAME-OF-YOUR-KEY>` and
   `<IP-OF-EC2-INSTANCE>`:
   ```bash
   chmod 400 <NAME-OF-YOUR-KEY>.pem
   ssh -i <NAME-OF-YOUR-KEY>.pem -N -L 8888:localhost:8888 ubuntu@<IP-OF-EC2-INSTANCE>
   ```
1. With the ssh port forward set up, select the **Outputs** tab, and click on the link corresponding
   to `WebAccessWithPortForwarding`. This will automatically log you into the server. Note that by
   default, the `WebAccess` link will not be accessible as the default security group prevents
   public http access.
1. Once logged in, click on `vision-system-data-demo.ipynb` from the left navigation bar to start
   the interactive demo

<div class="alert alert-block alert-info">
Note: At this point, you can start executing the rest of the notebook on your own jupyter instance.
</div>

Update your AWS CLI to enable support for vision system data.

In [ ]:
cd
curl "https://awscli.amazonaws.com/awscli-exe-linux-aarch64.zip" -o "awscliv2.zip"
unzip -q awscliv2.zip
sudo ./aws/install --update
if ! ( aws iotfleetwise create-decoder-manifest help | grep -q ros2 ); then
    echo "Error: The AWS CLI does not support the vision system data feature." >&2
    echo "       Delete any local model installed at ~/.aws/models/iotfleetwise" >&2
    kill -SIGINT $$
fi

## Set some global constants
These will be used throughout the demo.

First generate a unique id to avoid conflicts when creating resources:

In [ ]:
UUID="$(cat /proc/sys/kernel/random/uuid)"
DISAMBIGUATOR="${UUID:0:8}"

Then set variables with names that we are going to use throughout the demo:

In [ ]:
PREFIX="FW-VSD-ROS2-$DISAMBIGUATOR"
echo "Prefix: $PREFIX"

CRED_STACK_NAME="${PREFIX}-FwCredentialProviderStack"
CRED_ROLE_NAME="${PREFIX}-provider-role"
S3_BUCKET_NAME=`echo "${PREFIX}-bucket" | tr '[:upper:]' '[:lower:]'`

REGION='us-east-1'
SERVICE_PRINCIPAL="iotfleetwise.amazonaws.com"

SIGNAL_CATALOG_NAME="FW-VSD-ROS2-signal-catalog"
MODEL_MANIFEST_NAME="${PREFIX}-model"
DECODER_MANIFEST_NAME="${PREFIX}-decoder"
VEHICLE_NAME="${PREFIX}-vehicle"
FLEET_NAME="${PREFIX}-fleet"
CAMPAIGN_NAME="${PREFIX}-campaign"


ENDPOINT_URL_OPTION=""

LOG_DIR=~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data/logs
mkdir -p $LOG_DIR

## Getting the FWE binary

You can either download a pre-built binary, or build it from source.

<div class="alert alert-block alert-info">
Note: If you want to use your own custom ROS2 messages, you will need to <a href="#optional-building-from-source">build it from source</a>.
</div>

### Downloading the pre-built binary

**To quickly run the demo,** download the pre-built FWE binary from GitHub and install the required
runtime dependencies for the demo:

In [ ]:
cd ~/aws-iot-fleetwise-edge
mkdir -p build/iotfleetwise
curl -L https://github.com/aws/aws-iot-fleetwise-edge/releases/latest/download/aws-iot-fleetwise-edge-ros2-arm64.tar.gz -o aws-iot-fleetwise-edge-ros2-arm64.tar.gz
tar -zxf aws-iot-fleetwise-edge-ros2-arm64.tar.gz -C build/iotfleetwise/
sudo -H ./tools/install-deps-native.sh --with-ros2-support --runtime-only >  $LOG_DIR/deps.log 2>&1
source /opt/ros/galactic/setup.bash

### (Optional) Building from source
If you already downloaded the binary above, skip to the next section.
**Alternatively if you would like to build the FWE binary from source,** Uncomment the next two cells and execute them.

1. Install the dependencies for FWE with ROS2 support:

In [ ]:
# cd ~/aws-iot-fleetwise-edge
# sudo -H ./tools/install-deps-native.sh --with-ros2-support --prefix /usr/local
# sudo ldconfig

2. Compile FWE with ROS2 support:
<div class="alert alert-block alert-info">
   Note: If you want to use custom ROS2 messages, you need to add them in this step to the `colcon` build paths. For more info, refer to <a href="https://docs.ros.org/en/galactic/Tutorials/Beginner-Client-Libraries/Custom-ROS2-Interfaces.html">ROS2 documentation</a>.
</div>

In [ ]:
# # To build ROS2 with custom msgs, uncomment the next lines and point the path towards the package directory containing those msg definition.
# # For example, here we clone the CARLA message definitions and install the dependencies for them.
# # ROS2_CUSTOM_PKGS=~/ros-carla-msgs
# # if ! [ -d $ROS2_CUSTOM_PKGS ]; then git clone -b 1.3.0 https://github.com/carla-simulator/ros-carla-msgs.git $ROS2_CUSTOM_PKGS; fi
# # sudo apt install -y ros-galactic-ros-environment ros-galactic-diagnostic-msgs ros-galactic-ament-cmake-cppcheck ros-galactic-ament-cmake-cpplint ros-galactic-ament-cmake-uncrustify ros-galactic-ament-cmake-flake8 ros-galactic-ament-cmake-pep257 ros-galactic-ament-lint-auto
# source /opt/ros/galactic/setup.bash
# rm -rf build install log
# colcon build \
#   --paths ./ $ROS2_CUSTOM_PKGS \
#   --cmake-args --no-warn-unused-cli -DCMAKE_BUILD_TYPE=Release -DFWE_STATIC_LINK=On -DFWE_FEATURE_ROS2=On
# source install/setup.bash

## Set up the S3 bucket policy

If you haven't yet created a bucket for this demo, you can run the following cell to create it:

In [ ]:
aws s3 mb s3://$S3_BUCKET_NAME --region $REGION

Let's make sure the bucket has the necessary policy to allow FleetWise to write to it. First, we create the policy statement and save it to a file:

In [ ]:
if ! OWNERSHIP_CONTROLS=`aws s3api get-bucket-ownership-controls --bucket ${S3_BUCKET_NAME} 2> /dev/null` \
        || [ "`echo ${OWNERSHIP_CONTROLS} | jq -r '.OwnershipControls.Rules[0].ObjectOwnership'`" != "BucketOwnerEnforced" ]; then
        echo "Error: ACLs are enabled for bucket ${S3_BUCKET_NAME}. Disable them at https://s3.console.aws.amazon.com/s3/bucket/${S3_BUCKET_NAME}/property/oo/edit"
fi

In [ ]:
cd ~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data
cat << EOF > data/s3-bucket-policy.json
{
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "${SERVICE_PRINCIPAL}"
            },
            "Action": "s3:ListBucket",
            "Resource": "arn:aws:s3:::${S3_BUCKET_NAME}"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "${SERVICE_PRINCIPAL}"
            },
            "Action": ["s3:GetObject", "s3:PutObject"],
            "Resource": "arn:aws:s3:::${S3_BUCKET_NAME}/*"
        }
    ]
}
EOF
cat data/s3-bucket-policy.json | jq -C

{
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "gamma.iotfleetwise.aws.internal"
      },
      "Action": "s3:ListBucket",
      "Resource": "arn:aws:s3:::fw-vsd-ros2-db819165-bucket"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "gamma.iotfleetwise.aws.internal"
      },
      "Action": [
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Resource": "arn:aws:s3:::fw-vsd-ros2-db819165-bucket/*"
    }
  ]
}


Then we use this file to set the bucket policy 

In [ ]:
aws s3api put-bucket-policy --bucket $S3_BUCKET_NAME --policy file://data/s3-bucket-policy.json

## Set up the AWS IoT Role Alias

Run the following cell to launch a CloudFormation stack with the required IAM role and role alias for AWS IoT Credentials Provider in your AWS account.

<div class="alert alert-block alert-info">
Note: the S3 bucket must have been created in `us-east-1`, and the resources are created in AWS region `us-east-1`.
</div>


In [ ]:
cd ~/aws-iot-fleetwise-edge
aws cloudformation create-stack \
    --region $REGION \
    --stack-name "${CRED_STACK_NAME}" \
    --template-body file://tools/cfn-templates/iot-credentials-provider.yml \
    --parameters ParameterKey=RoleAlias,ParameterValue=${CRED_ROLE_NAME} \
        ParameterKey=S3BucketName,ParameterValue="${S3_BUCKET_NAME}" \
        ParameterKey=IoTCoreRegion,ParameterValue=${REGION} \
        ParameterKey=S3BucketPrefixPattern,ParameterValue='"*raw-data/${credentials-iot:ThingName}/*"' \
    --capabilities CAPABILITY_AUTO_EXPAND CAPABILITY_NAMED_IAM

In [ ]:
aws cloudformation --region $REGION wait stack-create-complete --stack-name "${CRED_STACK_NAME}"

## Provision and run FWE

1. Run the following to provision credentials for the vehicle.

<div class="alert alert-block alert-info">
   Note: The resources are created in AWS region `us-east-1`.
</div>

In [ ]:
BUILD_CONFIG_DIR=~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data/build_config
mkdir -p $BUILD_CONFIG_DIR
cd ~/aws-iot-fleetwise-edge

./tools/provision.sh \
    --vehicle-name $VEHICLE_NAME \
    --creds-role-alias ${CRED_ROLE_NAME} \
    --certificate-pem-outfile $BUILD_CONFIG_DIR/certificate.pem \
    --private-key-outfile $BUILD_CONFIG_DIR/private-key.key \
    --endpoint-url-outfile $BUILD_CONFIG_DIR/endpoint.txt \
    --vehicle-name-outfile $BUILD_CONFIG_DIR/vehicle-name.txt \
    --creds-role-alias-outfile $BUILD_CONFIG_DIR/creds-role-alias.txt \
    --creds-endpoint-url-outfile $BUILD_CONFIG_DIR/creds-endpoint.txt \
    --region $REGION


In [ ]:

./tools/configure-fwe.sh \
    --input-config-file configuration/static-config.json \
    --output-config-file $BUILD_CONFIG_DIR/config-0.json \
    --log-level Trace \
    --vehicle-name $VEHICLE_NAME \
    --endpoint-url `cat $BUILD_CONFIG_DIR/endpoint.txt` \
    --certificate-file `realpath $BUILD_CONFIG_DIR/certificate.pem` \
    --private-key-file `realpath $BUILD_CONFIG_DIR/private-key.key` \
    --creds-role-alias `cat $BUILD_CONFIG_DIR/creds-role-alias.txt` \
    --creds-endpoint-url `cat $BUILD_CONFIG_DIR/creds-endpoint.txt` \
    --persistency-path `realpath $BUILD_CONFIG_DIR` \
    --raw-data-buffer-size 2147483648

2. Run the following to start FWE. At this stage FWE will be waiting to receive a campaign from the
   cloud.

In [ ]:
# make sure to kill FWE if we already started it previously
if [ ! -z ${FWE_PID} ]; then kill ${FWE_PID}; wait ${FWE_PID}; fi

cd ~/aws-iot-fleetwise-edge
source /opt/ros/galactic/setup.bash
./build/iotfleetwise/aws-iot-fleetwise-edge $BUILD_CONFIG_DIR/config-0.json > $LOG_DIR/FWE.log 2>&1 &
FWE_PID=$!

## Playback ROS2 data

<div class="alert alert-block alert-info">
   Note:

   - For the next part, if you want to use your own ROS2 rosbag file, upload this to the EC2 instance and adjust the command below to play it back.
   - If you want to run the demo with the provided rosbag file, then simply execute the cell below as-is.

</div>

Data used in this demonstration was generated using the [CARLA](https://carla.org) simulator and recorded in ROS2 rosbag format.

Download and playback the ROS2 rosbag file containing example source data that includes video frame images, vehicle dynamics and perception data. Playback of the ~2 minute file is looped.

In [ ]:
cd ~/aws-iot-fleetwise-edge
aws s3 cp s3://aws-iot-fleetwise/rosbag2_vision_system_data_demo.db3 .

In [ ]:
# make sure to kill the playback if we already started it previously
if [ ! -z ${ROSBAG_PID} ]; then kill ${ROSBAG_PID}; wait ${ROSBAG_PID}; fi

source /opt/ros/galactic/setup.bash
ros2 bag play --loop rosbag2_vision_system_data_demo.db3 > $LOG_DIR/rosbag.log 2>&1 &
ROSBAG_PID=$!

2. **Optional:** If you are interested in viewing the example source data you can also download the file to your local machine and open it in [Foxglove Studio](https://foxglove.dev/).
![Foxglove studio screenshot](foxglove-studio-screenshot.jpg)

    * Download the ROS2 rosbag file: https://s3.console.aws.amazon.com/s3/object/aws-iot-fleetwise?region=us-east-1&prefix=rosbag2_vision_system_data_demo.db3
    * Download and install Foxglove Studio: https://foxglove.dev/download
    * In Foxglove Studio, open the ROS2 rosbag file:
        * Click 'Open local file'.
        * Select the `rosbag2_vision_system_data_demo.db3` file.
        * Click on the 'Layouts' tab on the left.
        * Click on the 'Import layout' button.
        * Select the file in this folder `vision-system-data-foxglove-layout.json`.
        * Click the 'Play' button to playback the recording.


## Creating the necessary files

In [ ]:
cd ~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data
script_dir=../../../tools/cloud

As we previously mentioned, to start collecting data, we need a few files:
  1. **ros2-config.json**: This file will be used to generate other files needed later for Signal Catalog and Decoder Manifest creation.
  1. **network-interfaces.json**: This file will be used to define the interface IDs and types of network interfaces for the decoder manifest
  
<div class="alert alert-block alert-info">
   Note:

   - For the next part, if you want to use your own ROS2 messages and you already built them above, you can edit the `ros2-config.json` file to add your topics, types and fully-qualified-names.
   - If you want to run the demo with the provided files, then simply execute the cells below with the paths provided.

</div>


### Create ros2-config.json

First, we need to create a JSON configuration file (`ros2-config.json`). 

* **fullyQualifiedName -** this name will be the name of the Sensor in the signal catalog (can be anything). In this config, we used example ROS2 topic name as a fullyQualifiedName with the Vehicle prefix.
* **interfaceId** - must match the interface ID used in the `static-config.json` file (on the Edge). In this example it's `"10"`.
* **topic** - topic name where message is published on
* **type** - the path to the message that is published

In [ ]:
ROS2_CONFIG_FILE="data/ros2-config.json"
cat $ROS2_CONFIG_FILE | jq -C

{
  "messages": [
    {
      "fullyQualifiedName": "Vehicle.Cameras.Front.Image",
      "interfaceId": "10",
      "topic": "/carla/ego_vehicle/rgb_front/image_compressed",
      "type": "sensor_msgs/msg/CompressedImage"
    },
    {
      "fullyQualifiedName": "Vehicle.Speed",
      "interfaceId": "10",
      "topic": "/carla/ego_vehicle/speedometer",
      "type": "std_msgs/msg/Float32"
    },
    {
      "fullyQualifiedName": "Vehicle.Airbag.CollisionIntensity",
      "interfaceId": "10",
      "topic": "/carla/ego_vehicle/collision_intensity",
      "type": "std_msgs/msg/Float32"
    },
    {
      "fullyQualifiedName": "Vehicle.Acceleration",
      "interfaceId": "10",
      "topic": "/carla/ego_vehicle/imu",
      "type": "sensor_msgs/msg/Imu"
    }
  ]
}


### Create network-interfaces.json

This file will be needed to generate input for **CreateDecoderManifest** API
<div class="alert alert-block alert-info">
   Note: It has to have the same **interfaceId** as provided before in the ros2-config.json. Other fields should remain the same for ros2-signals.

</div>

In [ ]:
NETWORK_INTERFACES_FILE="data/network-interfaces.json"
NETWORK_INTERFACES=`cat $NETWORK_INTERFACES_FILE`
echo $NETWORK_INTERFACES | jq -C

[
  {
    "interfaceId": "10",
    "type": "VEHICLE_MIDDLEWARE",
    "vehicleMiddleware": {
      "name": "ros2",
      "protocolName": "ROS_2"
    }
  }
]


### Generating CreateSignalCatalog input

The data collection starts in the modeling stage. FW signal catalog models signals in the VSS format. Therefore, the ROS2 message format requires some transformation before it can be imported into the service. 

We will use the previously created `ROS2_CONFIG_FILE` to generate this file.

In [ ]:
python3 $script_dir/ros2-to-nodes.py --config ${ROS2_CONFIG_FILE} --output data/ros2-nodes.json
ROS2_NODES=`cat data/ros2-nodes.json`
echo $ROS2_NODES | jq -C

[
  {
    "branch": {
      "fullyQualifiedName": "Types"
    }
  },
  {
    "struct": {
      "fullyQualifiedName": "Types.sensor_msgs_msg_CompressedImage"
    }
  },
  {
    "struct": {
      "fullyQualifiedName": "Types.std_msgs_Header"
    }
  },
  {
    "struct": {
      "fullyQualifiedName": "Types.builtin_interfaces_Time"
    }
  },
  {
    "property": {
      "fullyQualifiedName": "Types.builtin_interfaces_Time.sec",
      "dataType": "INT32",
      "dataEncoding": "TYPED"
    }
  },
  {
    "property": {
      "fullyQualifiedName": "Types.builtin_interfaces_Time.nanosec",
      "dataType": "UINT32",
      "dataEncoding": "TYPED"
    }
  },
  {
    "property": {
      "fullyQualifiedName": "Types.std_msgs_Header.stamp",
      "dataType": "STRUCT",
      "structFullyQualifiedName": "Types.builtin_interfaces_Time"
    }
  },
  {
    "property": {
      "fullyQualifiedName": "Types.std_msgs_Header.frame_id",
      "dataType": "STRING",
      "dataEncoding": "TYPED"
    }
  },
  {


The whole json file contains:

1. Three branch-nodes, representing ROS2 message.
2. Vision Data message is modeled as a `Sensor` with the `dataType` is `STRUCT`. `structFullyQualifiedName` points to another node which models the message's structure.
3. New branch for all ROS2 types (optional)
4. Three more nodes under every branch
5. For each struct, we declare property nodes.

### Create or update Signal Catalog

Now, using the generated JSON, e.g. `ros2-nodes.json`, we can create (or update the existing) signal catalog by running:

In [ ]:
echo "Checking for an existing signal catalog..."
SIGNAL_CATALOG_LIST=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise list-signal-catalogs`
SC_NAME=`echo ${SIGNAL_CATALOG_LIST} | jq -r .summaries[0].name`

if [ "${SC_NAME}" == "null" ]; then
    echo "No signal catalog found. Creating a new one..."
    SIGNAL_CATALOG_ARN=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-signal-catalog \
    --name "${SIGNAL_CATALOG_NAME}" \
    --nodes "${ROS2_NODES}" | jq -r .arn`
else
    echo "Found an existing catalog ${SC_NAME}. Updating it..."
    SIGNAL_CATALOG_NAME=${SC_NAME}
    SIGNAL_CATALOG_ARN=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise update-signal-catalog \
            --name "${SIGNAL_CATALOG_NAME}" \
            --description "ROS2 signals" \
            --nodes-to-update "${ROS2_NODES}" | jq -r .arn`
fi

echo $SIGNAL_CATALOG_ARN

To validate the signal catalog creation, run:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise list-signal-catalog-nodes --name "${SIGNAL_CATALOG_NAME}"

{
    "nodes": [
        {
            "branch": {
                "fullyQualifiedName": "Types"
            }
        },
        {
            "branch": {
                "fullyQualifiedName": "Vehicle",
                "description": "Vehicle"
            }
        },
        {
            "branch": {
                "fullyQualifiedName": "Vehicle.Airbag",
                "description": "Vehicle.Airbag"
            }
        },
        {
            "branch": {
                "fullyQualifiedName": "Vehicle.Cameras",
                "description": "Vehicle.Cameras"
            }
        },
        {
            "branch": {
                "fullyQualifiedName": "Vehicle.Cameras.Front",
                "description": "Vehicle.Cameras.Front"
            }
        },
        {
            "sensor": {
                "fullyQualifiedName": "Vehicle.Acceleration",
                "dataType": "STRUCT",
                "structFullyQualifiedName": "Types.sensor_msgs_msg_Imu"
            }
    

As a response, you will get the originally generated json with the parent field `nodes`.

### Create and activate Model Manifest

Model Manifest only requires a list of Sensors from the signal catalog. To simply filter these nodes out, run:

In [ ]:
NODE_LIST=`( echo ${ROS2_NODES} | jq -r '.[].sensor.fullyQualifiedName' | grep Vehicle\\. ) | jq -Rn '[inputs]'`
echo $NODE_LIST | jq -C

[
  "Vehicle.Cameras.Front.Image",
  "Vehicle.Speed",
  "Vehicle.Airbag.CollisionIntensity",
  "Vehicle.Acceleration"
]


Then create and activate Model Manifest:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-model-manifest \
    --name ${MODEL_MANIFEST_NAME} \
    --signal-catalog-arn ${SIGNAL_CATALOG_ARN} \
    --nodes "${NODE_LIST}"

MODEL_MANIFEST_ARN=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise update-model-manifest \
    --name ${MODEL_MANIFEST_NAME} \
    --status ACTIVE | jq -r .arn`

echo ${MODEL_MANIFEST_ARN}

{
    "arn": "arn:aws:iotfleetwise:eu-central-1:687737027363:model-manifest/ROS2-model-manifest"
}
arn:aws:iotfleetwise:eu-central-1:687737027363:model-manifest/ROS2-model-manifest


### Generating CreateDecoderManifest input

Decoder Manifest contains ROS2 message representation in a specific format. Decoder Manifest contains all crucial information about the message structure and field types that are used, which is required for decoding of the binary data transferred to the cloud service.

We will again use the previously created `ROS2_CONFIG_FILE` to generate this file.

In [ ]:
python3 $script_dir/ros2-to-decoders.py --config ${ROS2_CONFIG_FILE} --output data/ros2-decoders.json
ROS2_DECODERS=`cat data/ros2-decoders.json`
echo $ROS2_DECODERS | jq -C

[
  {
    "fullyQualifiedName": "Vehicle.Cameras.Front.Image",
    "type": "MESSAGE_SIGNAL",
    "interfaceId": "10",
    "messageSignal": {
      "topicName": "/carla/ego_vehicle/rgb_front/image_compressed:sensor_msgs/msg/CompressedImage",
      "structuredMessage": {
        "structuredMessageDefinition": [
          {
            "fieldName": "header",
            "dataType": {
              "structuredMessageDefinition": [
                {
                  "fieldName": "stamp",
                  "dataType": {
                    "structuredMessageDefinition": [
                      {
                        "fieldName": "sec",
                        "dataType": {
                          "primitiveMessageDefinition": {
                            "ros2PrimitiveMessageDefinition": {
                              "primitiveType": "INT32"
                            }
                          }
                        }
                      },
                      {
          

### Create and activate Decoder Manifest

Now you can call CreateDecoderManifest API by running

In [ ]:
DECODER_MANIFEST_ARN=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-decoder-manifest \
    --name ${DECODER_MANIFEST_NAME} \
    --model-manifest-arn ${MODEL_MANIFEST_ARN} \
    --network-interfaces "${NETWORK_INTERFACES}" \
    --signal-decoders "${ROS2_DECODERS}" | jq -r .arn`

echo ${DECODER_MANIFEST_ARN}

Now activate Decoder Manifest:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise update-decoder-manifest \
    --name ${DECODER_MANIFEST_NAME} \
    --status ACTIVE

 and wait for the status to get active:

In [ ]:
while true; do
    sleep 5
    DECODER_MANIFEST_SUMMARY=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise get-decoder-manifest --name ${DECODER_MANIFEST_NAME}`
    if [ `echo ${DECODER_MANIFEST_SUMMARY} | jq -r .status` == "ACTIVE" ]; then
        echo "Decoder is ACTIVE"
        break
    fi
done

### Create a fleet of vehicles

To deploy a campaign, we need a target for this campaign. This target can be a single vehicle, or more realistically, a fleet of vehicles. 

We have already provisioned one vehicle. This means we created an IoT Thing and the needed permissions and policies for it. 

1. Let's create a fleetwise vehicle using this IoT Thing:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-vehicle \
                --decoder-manifest-arn ${DECODER_MANIFEST_ARN} \
                --association-behavior ValidateIotThingExists \
                --model-manifest-arn ${MODEL_MANIFEST_ARN} \
                --vehicle-name "${VEHICLE_NAME}"

2. Then we create a fleet:

In [ ]:
FLEET_ARN=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-fleet \
    --fleet-id ${FLEET_NAME} \
    --description "Description is required" \
    --signal-catalog-arn ${SIGNAL_CATALOG_ARN} | jq -r .arn`

echo ${FLEET_ARN}

3. Finally, we associate the created vehicle(s) with the fleet:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise associate-vehicle-fleet \
                --fleet-id ${FLEET_NAME} \
                --vehicle-name "${VEHICLE_NAME}"

At this point, we are ready to define a campaign and deploy it to this fleet and all the vehicles associated with it

### Create and activate a campaign to collect ROS2 data

<div class="alert alert-block alert-info">
   Note:

   - For the next part, if you want to use your own ROS2 messages, edit the campaign files with the fully-qualified-names of your custom signals.
   - If you want to run the demo with the provided campaign files, then simply execute the cells below as they are.
   - The only supported destination for ROS2 data is S3.

</div>

The definition of campaign for ROS2 data is similar to telemetry data. For condition-based campaign, you can use every primitive value for the trigger. However, you can only collect the whole ROS2 message, partial collection is not supported. For example, you cannot collect only the `data` field of the CompressedImage message.

Example of a time-based campaign:

In [ ]:
cd ~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data
S3_CAMPAIGN_FILE="data/campaign-vision-system-data-heartbeat.json"
cat $S3_CAMPAIGN_FILE | jq -C

{
  "spoolingMode": "TO_DISK",
  "collectionScheme": {
    "timeBasedCollectionScheme": {
      "periodMs": 10000
    }
  },
  "signalsToCollect": [
    {
      "name": "Vehicle.Cameras.Front.Image"
    },
    {
      "name": "Vehicle.Speed"
    },
    {
      "name": "Vehicle.Airbag.CollisionIntensity"
    },
    {
      "name": "Vehicle.Acceleration"
    }
  ]
}


Example condition-based campaign:

In [ ]:
cat data/campaign-brake-event-vision-system-data.json | jq -C

{
  "compression": "SNAPPY",
  "diagnosticsMode": "OFF",
  "spoolingMode": "TO_DISK",
  "collectionScheme": {
    "conditionBasedCollectionScheme": {
      "conditionLanguageVersion": 1,
      "expression": "$variable.`Vehicle.ABS.DemoBrakePedalPressure` > 7000",
      "minimumTriggerIntervalMs": 1000,
      "triggerMode": "ALWAYS"
    }
  },
  "postTriggerCollectionDuration": 1000,
  "signalsToCollect": [
    {
      "name": "Vehicle.ECM.DemoEngineTorque"
    },
    {
      "name": "Vehicle.ABS.DemoBrakePedalPressure"
    },
    {
      "name": "Vehicle.Cameras.Front.Image"
    },
    {
      "name": "Vehicle.Speed"
    },
    {
      "name": "Vehicle.Airbag.CollisionIntensity"
    },
    {
      "name": "Vehicle.Acceleration"
    }
  ]
}


For this demo, let's use the time-based campaign.

Let's first populate the campaign definition with the correct signal catalog ARN and fleet ARN:

In [ ]:
CAMPAIGN=`cat ${S3_CAMPAIGN_FILE} \
        | jq .name=\"${CAMPAIGN_NAME}\" \
        | jq .signalCatalogArn=\"${SIGNAL_CATALOG_ARN}\" \
        | jq .targetArn=\"${FLEET_ARN}\"`
echo $CAMPAIGN | jq -C

{
  "spoolingMode": "TO_DISK",
  "collectionScheme": {
    "timeBasedCollectionScheme": {
      "periodMs": 10000
    }
  },
  "signalsToCollect": [
    {
      "name": "Vehicle.Cameras.Front.Image"
    },
    {
      "name": "Vehicle.Speed"
    },
    {
      "name": "Vehicle.Airbag.CollisionIntensity"
    },
    {
      "name": "Vehicle.Acceleration"
    }
  ],
  "name": "FW-VSD-ROS2-fleet",
  "signalCatalogArn": "arn:aws:iotfleetwise:eu-central-1:687737027363:signal-catalog/FW-VSD-ROS2-signal-catalog",
  "targetArn": "arn:aws:iotfleetwise:eu-central-1:687737027363:fleet/FW-VSD-ROS2-fleet"
}


To deploy it to the vehicle, run:

<div class="alert alert-block alert-info">
Note: to collect data in the json format, uncomment the second line.
</div>

In [ ]:
DATA_FORMAT="PARQUET"
# DATA_FORMAT="JSON"

aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise create-campaign \
    --cli-input-json "${CAMPAIGN}" \
    --data-destination-configs "[{\"s3Config\":{\"bucketArn\":\"arn:aws:s3:::${S3_BUCKET_NAME}\",\"prefix\":\"${CAMPAIGN_NAME}-s3\",\"dataFormat\":\"${DATA_FORMAT}\",\"storageCompressionFormat\":\"NONE\"}}]"| jq -r .arn

Wait for the campaign status to transition to `WAITING_FOR_APPROVAL`:

In [ ]:
while true; do
    sleep 5
    CAMPAIGN_STATUS=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise get-campaign --name ${CAMPAIGN_NAME}`
    if [ `echo ${CAMPAIGN_STATUS} | jq -r .status` == "WAITING_FOR_APPROVAL" ]; then
        echo "Campaign status is WAITING_FOR_APPROVAL"
        break
    fi
done

After successful deployment, approve your campaign by running:

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise update-campaign --name ${CAMPAIGN_NAME} --action APPROVE

To verify that the campaign was deployed to the vehicle, we wait until the vehicle status transitions from `READY` to `HEALTHY`

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise get-vehicle-status --vehicle-name "${VEHICLE_NAME}"

In [ ]:
while true; do
    VEHICLE_STATUS=`aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise get-vehicle-status --vehicle-name "${VEHICLE_NAME}"`
    for ((l=0; ; l++)); do
        C_NAME=`echo ${VEHICLE_STATUS} | jq -r .campaigns[${l}].campaignName`
        C_STATUS=`echo ${VEHICLE_STATUS} | jq -r .campaigns[${l}].status`
        echo "${C_NAME} is ${C_STATUS}"
        # If the campaign was not found (when the index is out-of-range jq will return 'null')
        if [ "${C_NAME}" == "null" ]; then
            echo "Error: Campaign not found in vehicle status for vehicle $1" >&2
            kill -SIGINT $$
        # If the campaign was found \
        elif [ "${C_NAME}" == "${CAMPAIGN_NAME}" ]; then
            if [ "${C_STATUS}" == "HEALTHY" ]; then
                break 2
            fi
            break
        fi
    done
    sleep 5
done

After campaign becomes active, FWE will start collecting ROS2 data.

<div class="alert alert-block alert-info">
   Note: It can take up to 15 minutes for data to start appearing in the S3 bucket.
</div>

### Inspecting collected data

After the data was collected and uploaded to S3 from Edge, FW processing pipeline will start working on transforming it. The buffering can take up to 15 min. By the end of processing, you will see 3 folders in your campaign bucket:

- The content of **raw-data/** is uploaded directly by the FWE. This data is available in the near real time after Edge collection. Collected ROS2 message data is transferred as CDR binary (signal_byte_values).

- **processed-data/** folder contains data decoded by the FW pipeline. Data is partitioned by the timestamp in the subfolders.

- **unstructured-data/** folder contains extracted binary field data as files.

*Note: since **sensor_msgs_msg_CompressedImage.data** field was marked as binary, you can see a link to the binary data attached, which you can find in the **unstructured-data/** folder.*


In [ ]:
S3_URL="s3://${S3_BUCKET_NAME}/${CAMPAIGN_NAME}-s3/processed-data/"

while ! COLLECTED_FILES=`aws s3 ls --recursive ${S3_URL}`; do
    sleep 60
done


<div class="alert alert-block alert-info">
   Note: It can take up to 15 minutes for data to start appearing in the S3 bucket.
</div>

If the bucket did contain data, let's download some of them:

In [ ]:
cd ~/aws-iot-fleetwise-edge/docs/dev-guide/vision-system-data

NUM_FILES=2
COLLECTED_DATA_DIR="collected-data/"
mkdir -p ${COLLECTED_DATA_DIR}

echo "${COLLECTED_FILES}" | head -n${NUM_FILES} | while read LINE; do
    KEY=`echo ${LINE} | cut -d ' ' -f4`
    aws s3 cp s3://${S3_BUCKET_NAME}/${KEY} ${COLLECTED_DATA_DIR}
done

Now let's visualize some of the data by using a helper script. The following example will plot all `x`, `y` and `z` values of `Vehicle.Acceleration.angular_velocity`. Please note the full `Vehicle.Acceleration` struct was collected, but here we are only visualizing part of it.

In [ ]:
echo "Converting from Firehose ${DATA_FORMAT} to HTML..."
OUTPUT_FILE_HTML="${COLLECTED_DATA_DIR}${VEHICLE_NAME}-${DATA_FORMAT}-acceleration.html"
extension=`echo "${DATA_FORMAT}" | tr '[:upper:]' '[:lower:]'`
python3 $script_dir/firehose-to-html.py \
    --vehicle-name ${VEHICLE_NAME} \
    --files ${COLLECTED_DATA_DIR}*.${extension} \
    --include-signals 'Vehicle.Acceleration.angular_velocity.' \
    --html-filename ${OUTPUT_FILE_HTML}


In [ ]:
cat ${OUTPUT_FILE_HTML} | displayHTML

Similarly we can visualize other signals. For example, the following plots the `Vehicle.Speed` signal, which contains only the `data` value. We also include the `Image.data` signal so that we can extract the S3 filenames to download next.

In [ ]:
echo "Converting from Firehose ${DATA_FORMAT} to HTML..."
OUTPUT_FILE_HTML="${COLLECTED_DATA_DIR}${VEHICLE_NAME}-${DATA_FORMAT}.html"
OUTPUT_FILE_S3_LINKS="${COLLECTED_DATA_DIR}${VEHICLE_NAME}-s3-links-${DATA_FORMAT}.txt"
extension=`echo "${DATA_FORMAT}" | tr '[:upper:]' '[:lower:]'`
python3 $script_dir/firehose-to-html.py \
    --vehicle-name ${VEHICLE_NAME} \
    --files ${COLLECTED_DATA_DIR}*.${extension} \
    --include-signals 'Vehicle.Speed.data,Image.data' \
    --s3-links-filename ${OUTPUT_FILE_S3_LINKS} \
    --html-filename ${OUTPUT_FILE_HTML}


In [ ]:
cat ${OUTPUT_FILE_HTML} | displayHTML

Then we can download some of the images referenced in the processed files. Please note that we collect a smaller number of image samples than other signals. You can see the timestamp in the filename to correlate with the graphs above.

In [ ]:
if [ -s ${OUTPUT_FILE_S3_LINKS} ]; then
    echo "Downloading the first 10 linked files..."
    i=0
    cat ${OUTPUT_FILE_S3_LINKS} | while read LINE; do
        if ((i < 10)); then
        IMAGE_FILE=`basename ${LINE}.jpg`
        # Remove random prefix so that filenames begin with date
        IMAGE_FILE=`echo ${IMAGE_FILE} | sed -E 's/^[0-9a-f]{8}\-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{12}\-//'`
            aws s3 cp ${LINE} ${COLLECTED_DATA_DIR}${IMAGE_FILE}
        fi
        i=$((i+1))
    done
fi

## Clean up

<div class="alert alert-block alert-info">
   Note: If you clean-up the resources it will cancel any ongoing campaign.
</div>

### Suspend and delete the campaigns

In [ ]:
echo "Suspending campaign..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise update-campaign \
    --name ${CAMPAIGN_NAME} \
    --action SUSPEND

echo "Deleting campaign..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-campaign \
    --name ${CAMPAIGN_NAME}

### Disassociate the vehicle(s) from fleet

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise disassociate-vehicle-fleet \
    --fleet-id ${FLEET_NAME} \
    --vehicle-name "${VEHICLE_NAME}"

### Delete the vehicle

In [ ]:
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-vehicle \
    --vehicle-name "${VEHICLE_NAME}"

{
    "vehicleName": "FW-VSD-ROS2-vehicle",
    "arn": "arn:aws:iotfleetwise:eu-central-1:687737027363:vehicle/FW-VSD-ROS2-vehicle"
}


### Delete the fleet

In [ ]:
echo "Deleting fleet..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-fleet \
    --fleet-id ${FLEET_NAME}

### Delete the decoder manifest

In [ ]:
echo "Deleting decoder manifest..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-decoder-manifest \
    --name ${DECODER_MANIFEST_NAME}

### Delete the model manifest

In [ ]:
echo "Deleting model manifest..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-model-manifest \
    --name ${MODEL_MANIFEST_NAME}

### Delete the signal catalog

In [ ]:
echo "Deleting signal catalog..."
aws ${ENDPOINT_URL_OPTION} --region $REGION iotfleetwise delete-signal-catalog \
    --name ${SIGNAL_CATALOG_NAME}

### Clean up all resources created by the `provision.sh` script:

In [ ]:
cd ~/aws-iot-fleetwise-edge
tools/provision.sh \
    --vehicle-name ${VEHICLE_NAME} \
    --region $REGION \
    --only-clean-up

### (Optional) Clean up the S3 bucket

<div class="alert alert-block alert-info">
Note: If you don't need the bucket/folder anymore, uncomment the lines below to delete them.
</div>

In [ ]:
# Uncomment to delete the folder
#aws s3 rm s3://${S3_BUCKET_NAME}/${CAMPAIGN_NAME}-s3 --recursive --quiet --region $REGION

# Uncomment to delete the bucket itself
#aws s3api delete-bucket --bucket ${S3_BUCKET_NAME} --region $REGION

### Delete the AWS CloudFormation stack for the AWS IoT Credentials Provider role alias, where `<CRED_STACK_NAME>` was your chosen stack name:

In [ ]:
aws cloudformation delete-stack --region $REGION --stack-name "${CRED_STACK_NAME}" 

### Kill the background process that replays the rosbag and the FWE process

In [ ]:
kill $ROSBAG_PID
kill $FWE_PID